# 1、查看数据集

In [7]:
# 查看数据集--小莎士比亚训练集
with open('input.txt', 'r') as  f:
    text = f.read()

data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


# 2、测试分词工具

In [8]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens[:24]) # 打印分词后对应字典中的索引

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


# 3、将分词后的文本变成小批量输入到模型
- 创建小批量输入
- 创建对应的标签，也就是向后错一位的输入x

In [9]:
import torch
buf = torch.tensor(tokens[:24 + 1])     # 先转成tensor格式
x = buf[:-1].view(4, 6)                 # 小批量的输入，直接调整张量形状就可以做到分出批量了
y = buf[1:].view(4, 6)                  # 小批量对应的输入对应的标签
print(x)
print(y)

tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])


# 4、测试分词并分成小批量的文本，输入模型后的反应

In [10]:
from GPT2 import GPT

In [11]:
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

In [12]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

# 准备小批量数据
import tiktoken

enc = tiktoken.get_encoding('gpt2')
with open('input.txt', 'r') as  f:
    text = f.read()

data = text[:1000]
tokens = enc.encode(data)

B, T = 4, 32            # 批量为4， 序列长度为32

import torch
buf = torch.tensor(tokens[:B*T + 1])     # 先转成tensor格式
x = buf[:-1].view(B, T)                 # 小批量的输入，直接调整张量形状就可以做到分出批量了
y = buf[1:].view(B, T) 

x = x.to(device)
y = y.to(device)


In [13]:
# 查看输出和损失是否正常
model = GPT(GPTConfig)
model.to(device)
logits, loss  = model(x, y)
 
print(logits.shape)
print(loss)

torch.Size([4, 32, 50257])
tensor(11.0398, device='cuda:0', grad_fn=<NllLossBackward0>)


c:\deeplearn\Project\2_Myself  project\28_GPT-2\GPT2.py:59: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  y = F.scaled_dot_product_attention(q, k, v, is_causal=True)


In [14]:
print("Device name:", torch.cuda.get_device_properties('cuda').name)
print("FlashAttention available:", torch.backends.cuda.flash_sdp_enabled())
print(f'torch version: {torch.version}')

Device name: NVIDIA GeForce RTX 3090
FlashAttention available: True
torch version: <module 'torch.version' from 'd:\\miniconda\\envs\\gpt2\\Lib\\site-packages\\torch\\version.py'>
